In [ ]:
import query_helpers

from pathlib import PurePath
from raster_compare.base import RasterFile
import os
import numpy as np
import pandas as pd

# 50 m

In [ ]:
SNOW_DEPTH_DIR = HOME_DIR = PurePath(f"{os.environ['HOME']}/skiles-group1/ERW-SfM/50m")


aso_snow_depth = RasterFile(
    SNOW_DEPTH_DIR / '20180524_ASO_snow_depth_50m.tif',
    band_number=1
)
aso_snow_depth_values = aso_snow_depth.band_values()
np.ma.masked_where(
    aso_snow_depth_values <= 0.0,
    aso_snow_depth_values,
    copy=False
)

sfm_snow_depth = RasterFile(
    SNOW_DEPTH_DIR / '20180524_Agisoft_snow_depth_50m.tif',
    band_number=1
)
assert aso_snow_depth.geo_transform == sfm_snow_depth.geo_transform

sfm_snow_depth_values = sfm_snow_depth.band_values()
np.ma.masked_where(
    aso_snow_depth_values.mask,
    sfm_snow_depth_values,
    copy=False
);

## Prepare

In [ ]:
df = pd.DataFrame({
    'aso_snow_depth': aso_snow_depth_values.ravel(),
    'sfm_snow_depth': sfm_snow_depth_values.ravel(),
})
df.dropna(inplace=True, how='all')

### Depth Differences, Volume, and SWE 

In [ ]:
df = query_helpers.diff_vol_swe(df, aso_snow_depth, sfm_snow_depth)

In [ ]:
positive_sfm = query_helpers.get_positive(df, 'sfm_snow_depth')
negative_sfm = query_helpers.get_negative(df, 'sfm_snow_depth')
no_values_sfm = query_helpers.get_no_data(df, 'sfm_snow_depth')

columns = ['sfm_snow_depth', 'aso_snow_depth', 'sd_difference']
pd.set_option('display.float_format', lambda x: '%.2f m' % x)

# Snow Depth (50 m)

In [ ]:
df[columns].agg([np.mean, np.median, np.std])

### Positive SfM values

In [ ]:
positive_sfm[columns].agg([np.mean, np.median, np.std])

### Negative SfM values

In [ ]:
negative_sfm[columns].agg([np.mean, np.median, np.std])

In [ ]:
pd.set_option('display.float_format', None)

# Snow Volume (50 m)

In [ ]:
m2_style = "{:,.0f} m<sup>3</sup>"

table_style = {
    'sfm_snow_volume': m2_style,
    'aso_snow_volume': m2_style,
    'difference': m2_style,
    'sfm % to aso': "{:.2%}",
    'percent_sfm_scene': "{:.2%}",
    'percent_aso_scene': "{:.2%}",
}
columns = ['casi_class', 'aso_snow_volume', 'sfm_snow_volume']

In [ ]:
total_volume = pd.DataFrame({
    'sfm_snow_volume': df.sfm_snow_volume.sum(),
    'aso_snow_volume': df.aso_snow_volume.sum(),
    },
    index=[0]
)
total_volume['difference'] = total_volume.aso_snow_volume - total_volume.sfm_snow_volume
total_volume['sfm % to aso'] = total_volume.sfm_snow_volume / total_volume.aso_snow_volume
total_volume.style.format(table_style)

In [ ]:
captured_volume = pd.DataFrame({
    'sfm_snow_volume': positive_sfm.sfm_snow_volume.sum(),
    'aso_snow_volume': positive_sfm.aso_snow_volume.sum(),
    },
    index=[0]
)
captured_volume['difference'] = captured_volume.aso_snow_volume - captured_volume.sfm_snow_volume
captured_volume['sfm % to aso'] = captured_volume.sfm_snow_volume / captured_volume.aso_snow_volume
captured_volume.style.format(table_style)

In [ ]:
missed_volume = pd.DataFrame({
    'sfm_snow_volume': negative_sfm.sfm_snow_volume.sum(),
    'aso_snow_volume': negative_sfm.aso_snow_volume.sum(),
    },
    index=[0]
)
missed_volume['difference'] = missed_volume.aso_snow_volume - missed_volume.sfm_snow_volume
missed_volume['sfm % to aso'] = missed_volume.sfm_snow_volume / missed_volume.aso_snow_volume
missed_volume.style.format(table_style)

In [ ]:
assert (captured_volume.aso_snow_volume + missed_volume.aso_snow_volume).item(), total_volume.aso_snow_volume.item()

In [ ]:
num_pixels_aso = df.aso_snow_depth.count()
num_pixels_sfm = positive_sfm.aso_snow_depth.count()

In [ ]:
print("Pixels with depth:")
print(f"  ASO count: {num_pixels_aso:,}")
print(f"  SfM count: {num_pixels_sfm:,}")
print("Percent ASO pixels:")
print(f"  with values in SfM: {num_pixels_sfm/num_pixels_aso:.2%}")
print(f"  with negative in SfM: {negative_sfm.query('sfm_snow_depth == sfm_snow_depth').aso_snow_depth.count()/num_pixels_aso:.2%}")
print(f"  with no value in SfM: {no_values_sfm.aso_snow_depth.count()/num_pixels_aso:.2%}")

# SWE (50 m)

In [ ]:
m2_style = "{:,.0f} m<sup>3</sup>"

table_style = {
    'sfm_swe': m2_style,
    'aso_swe': m2_style,
    'difference': m2_style,
    'sfm % to aso': "{:.2%}",
}
columns = ['aso_swe', 'sfm_swe']

In [ ]:
total_swe = pd.DataFrame({
    'sfm_swe': df.sfm_swe.sum(),
    'aso_swe': df.aso_swe.sum(),
    },
    index=[0]
)
total_swe['difference'] = total_swe.aso_swe - total_swe.sfm_swe
total_swe['sfm % to aso'] = total_swe.sfm_swe / total_swe.aso_swe
total_swe.style.format(table_style)

## NSIDC SWE 

In [ ]:
swe_aso_nsidc = RasterFile(
    SNOW_DEPTH_DIR / '20180524_ASO_SWE_ERW_basin_50m.tif',
    band_number=1
)
swe_aso_values = np.ma.masked_where(
    aso_snow_depth_values.mask,
    swe_aso_nsidc.band_values(),
    copy=False
)

In [ ]:
np.median((swe_aso_values / aso_snow_depth_values).compressed())

In [ ]:
swe_aso_values.sum()